In [ ]:
from aes_hw import AES
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_OLED
from pynq import Overlay
import time

In [2]:
def pad_text(text):
    """Pad text to ensure it is 16 characters long."""
    return text.ljust(16, ' ')

In [3]:
def oled_display(statement, overlay_path="base.bit", pmod_port="PMODB"):

    # Load the base overlay
    base = BaseOverlay(overlay_path)
    
    # Initialize the Pmod OLED
    pmod_oled = Pmod_OLED(getattr(base, pmod_port))
    
    # Display the statement on the OLED
    pmod_oled.clear()  # Clear the OLED screen
    pmod_oled.write(str(statement))  # Write the statement to the OLED

In [4]:
# Input the key from the user (<=16-character string)
key_string = input("Enter a key (<=16 characters): ")
key_string = pad_text(key_string)  # Pad key if it's shorter than 16 characters
    
# Convert the padded key string to a 16-byte integer
master_key = int.from_bytes(key_string.encode('utf-8'), byteorder='big')
aes = AES(master_key)

# Input the plaintext message (<=16 characters string)
original_message = input("Enter a plaintext message (<=16 characters): ")
padded_message = pad_text(original_message)  # Pad message if it's shorter than 16 characters

print(f"Original message (padded if needed): {padded_message}")

Enter a key (<=16 characters): sr
Enter a plaintext message (<=16 characters): sr
Original message (padded if needed): sr              


In [ ]:
oled_display(f"input message: \n\n{original_message}")

In [5]:
overlay = Overlay("mix_imix.bit")

In [ ]:
# Convert the padded plaintext string to a 16-byte integer
plaintext = int.from_bytes(padded_message.encode('utf-8'), byteorder='big')
#   start_time  
start_time = time.time()
# Encrypt the plaintext
encrypted = aes.encrypt(plaintext)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken for encryption with hw acceleration: {elapsed_time:.9f} seconds")    
# Convert the encrypted integer to bytes and print in hex format
encrypted_bytes = encrypted.to_bytes(16, byteorder='big')
encrypted_hex = encrypted_bytes.hex()
    
# Convert the encrypted bytes to a string (hex encoded) for visual representation
encrypted_string = encrypted_bytes.decode('latin-1')
    
# Print the encrypted output in hex and string format
print(f"Encrypted output (hex): {encrypted_hex}")
print(f"Encrypted output (string): {encrypted_string}")
    

Entering __round_encrypt...
Time taken for encryption: 0.007611275 seconds
Encrypted output (hex): ae69f35902e74ac9d6a05e9d1ae3b75f
Encrypted output (string): ®ióYçJÉÖ ^ã·_


In [ ]:
oled_display(f"encryption done\ncipher text: \n{encrypted_string}")

In [ ]:
overlay = Overlay("mix_imix.bit")

In [ ]:
# Ask the user to enter the key again for decryption
reentered_key = input("Enter the key again for decryption: ")
reentered_key = pad_text(reentered_key)  # Pad reentered key if it's shorter than 16 characters
    
# Check if the reentered key matches the original key
if reentered_key != key_string:
    print("Decryption failed: Incorrect key.")
    oled_display(f"Decrypt failed\nIncorrect Key")
else:
    # Convert the reentered key to a 16-byte integer for decryption
    master_key_for_decryption = int.from_bytes(reentered_key.encode('utf-8'), byteorder='big')
    aes_for_decryption = AES(master_key_for_decryption)
    
    start_time = time.time()
    # Decrypt the ciphertext
    decrypted = aes_for_decryption.decrypt(encrypted)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for decryption with hw acceleration: {elapsed_time:.9f} seconds")
    
    # Convert the decrypted integer back to a string (16-character)
    decrypted_message = decrypted.to_bytes(16, byteorder='big').decode('utf-8')
    
    # Unpad the decrypted message (strip any spaces added during padding)
    decrypted_message = decrypted_message.rstrip()

    # Print the decrypted message
    print(f"Decrypted output: {decrypted_message}")
    oled_display(f"decryption done\nplain text: \n{decrypted_message}")

Enter the key again for decryption: sr
Time taken for decryption: 0.004275799 seconds
Decrypted output: sr
